<a href="https://colab.research.google.com/github/bewithankit/CS3DP19/blob/main/PySpark_vs_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
import time

# Initialize Spark
conf = SparkConf().setAppName('wordCount')
sc = SparkContext(conf=conf)

# Start timing
start_time = time.time()

# Read the text file and perform word count
linesRDD = sc.textFile("/content/ulysses.txt")
wordsRDD = linesRDD.flatMap(lambda line: line.split())
wordCounts = wordsRDD.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Collect the results
word_counts_collected = wordCounts.collect()

# Stop timing
end_time = time.time()

# Print the word counts
print("\nWord count:\n", word_counts_collected)

# Print the time taken
print("Time taken with PySpark: {} seconds".format(end_time - start_time))

# Stop Spark
sc.stop()


Word count:
 [('The', 1035), ('Project', 79), ('of', 8136), ('Ulysses', 6), ('ebook', 2), ('is', 1242), ('use', 42), ('anyone', 17), ('anywhere', 11), ('in', 4612), ('United', 19), ('States', 7), ('other', 271), ('world', 92), ('at', 1215), ('no', 363), ('restrictions', 2), ('whatsoever.', 2), ('may', 71), ('it,', 116), ('give', 121), ('away', 115), ('re-use', 2), ('this', 379), ('online', 4), ('are', 467), ('have', 620), ('check', 5), ('country', 42), ('where', 204), ('before', 182), ('using', 8), ('eBook.', 2), ('Author:', 1), ('date:', 1), ('July', 1), ('1,', 3), ('2003', 1), ('[eBook', 1), ('March', 5), ('Language:', 1), ('English', 19), ('EBook', 1), ('produced', 14), ('***', 4), ('OF', 36), ('GUTENBERG', 3), ('ULYSSES', 2), ('Contents', 1), ('[', 36), ('1', 53), ('II', 2), ('4', 21), ('8', 9), ('9', 14), ('10', 17), ('12', 6), ('14', 9), ('III', 2), ('16', 11), ('17', 5), ('came', 187), ('stairhead,', 1), ('bowl', 11), ('lather', 3), ('mirror', 31), ('razor', 5), ('yellow', 39),

In [5]:
from dask.distributed import Client
import dask.bag as db
import time

# Initialize Dask Client
client = Client()

# Start timing
start_time = time.time()

# Read the text file into a Dask Bag and perform word count
lines = db.read_text("/content/ulysses.txt")
word_counts = lines.map(str.split).flatten().frequencies(sort=True)

# Compute the results
word_counts_computed = word_counts.compute()

# Stop timing
end_time = time.time()

# Print the word counts
print(word_counts_computed)

# Print the time taken
print("Time taken with Dask: {} seconds".format(end_time - start_time))

# Shut down the Dask client
client.shutdown()


INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34215
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40877'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41021'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:35353', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35353
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:50650
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:46021', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46021
INFO:distributed.core:Sta

[('the', 13613), ('of', 8136), ('and', 6541), ('a', 5839), ('to', 4788), ('in', 4612), ('his', 3034), ('he', 2712), ('I', 2429), ('with', 2391), ('that', 2167), ('was', 2006), ('on', 1894), ('for', 1791), ('it', 1680), ('her', 1505), ('you', 1368), ('is', 1242), ('at', 1215), ('by', 1177), ('him', 1112), ('as', 1099), ('all', 1040), ('The', 1034), ('from', 1012), ('or', 938), ('He', 908), ('be', 821), ('she', 768), ('they', 768), ('had', 766), ('out', 760), ('not', 744), ('my', 708), ('Mr', 699), ('their', 678), ('up', 661), ('like', 649), ('me', 640), ('have', 620), ('an', 591), ('A', 560), ('one', 499), ('them', 497), ('And', 497), ('about', 493), ('when', 482), ('said.', 480), ('were', 473), ('are', 467), ('what', 458), ('which', 457), ('your', 451), ('says', 450), ('so', 450), ('if', 438), ('there', 428), ('Bloom', 428), ('but', 424), ('said', 423), ('old', 419), ('over', 392), ('this', 379), ('down', 367), ('no', 363), ('would', 349), ('then', 347), ('after', 345), ('who', 342), (

INFO:distributed.scheduler:Scheduler closing due to unknown reason...
INFO:distributed.scheduler:Scheduler closing all comms


In [9]:
from pyspark import SparkContext, SparkConf
import time
import re

def tokenize(text):
    """Use a regular expression to tokenize the text on word boundaries."""
    return re.findall(r'\w+', text.lower())

# Initialize Spark
conf = SparkConf().setAppName('wordCount')
sc = SparkContext(conf=conf)

# Start timing
start_time = time.time()

# Read the text file and perform word count
linesRDD = sc.textFile("/content/ulysses.txt")
wordsRDD = linesRDD.flatMap(tokenize)
wordCounts = wordsRDD.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Collect the results
word_counts_collected = wordCounts.collect()

# Stop timing
end_time = time.time()

# Print the word counts
print("\nWord count:\n", word_counts_collected)

# Print the time taken
print("Time taken with PySpark: {} seconds".format(end_time - start_time))

# Stop Spark
sc.stop()



Word count:
 [('project', 94), ('gutenberg', 98), ('ebook', 14), ('of', 8263), ('this', 490), ('is', 1455), ('use', 52), ('anyone', 21), ('anywhere', 12), ('in', 4991), ('united', 22), ('other', 337), ('world', 155), ('at', 1301), ('no', 691), ('restrictions', 2), ('whatsoever', 7), ('may', 108), ('give', 176), ('away', 182), ('online', 4), ('www', 9), ('org', 9), ('are', 556), ('have', 701), ('check', 5), ('country', 58), ('where', 335), ('before', 222), ('using', 8), ('title', 8), ('james', 35), ('july', 2), ('1', 140), ('2003', 1), ('4300', 1), ('language', 46), ('credits', 1), ('produced', 18), ('choat', 1), ('start', 17), ('contents', 4), ('i', 2989), ('4', 34), ('8', 23), ('9', 24), ('10', 24), ('12', 8), ('14', 10), ('iii', 2), ('16', 13), ('17', 7), ('buck', 116), ('mulligan', 163), ('came', 197), ('bowl', 14), ('lather', 4), ('mirror', 43), ('razor', 7), ('crossed', 26), ('yellow', 50), ('was', 2133), ('sustained', 11), ('gently', 38), ('he', 4221), ('aloft', 5), ('_introibo'

In [10]:
from dask.distributed import Client
import dask.bag as db
import time
import re

def tokenize(text):
    """Use a regular expression to tokenize the text on word boundaries."""
    return re.findall(r'\w+', text.lower())

# Initialize Dask Client
client = Client()

# Start timing
start_time = time.time()

# Read the text file into a Dask Bag and perform word count
lines = db.read_text("/content/ulysses.txt")
word_counts = lines.map(tokenize).flatten().frequencies(sort=True)

# Compute the results
word_counts_computed = word_counts.compute()

# Stop timing
end_time = time.time()

# Print the word counts
print(word_counts_computed)

# Print the time taken
print("Time taken with Dask: {} seconds".format(end_time - start_time))

# Shut down the Dask client
client.shutdown()


INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43663
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35487'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43667'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:43781', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:43781
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:42956
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33647', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:33647
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:42948
INFO:distributed.scheduler:Receive client connection: Client-a1639de9-7d63-11ee-81b8-0242ac1c000c

[('the', 15092), ('of', 8263), ('and', 7273), ('a', 6565), ('to', 5036), ('in', 4991), ('he', 4221), ('his', 3328), ('i', 2989), ('s', 2831), ('that', 2791), ('_', 2720), ('with', 2562), ('it', 2523), ('was', 2133), ('on', 2126), ('you', 2087), ('for', 1959), ('her', 1783), ('him', 1524), ('is', 1455), ('all', 1336), ('at', 1301), ('by', 1293), ('said', 1208), ('as', 1196), ('she', 1188), ('from', 1102), ('they', 1051), ('or', 1033), ('bloom', 999), ('what', 973), ('me', 944), ('not', 919), ('out', 898), ('be', 894), ('my', 832), ('up', 830), ('had', 814), ('there', 790), ('one', 741), ('like', 731), ('their', 719), ('mr', 717), ('have', 701), ('but', 696), ('no', 691), ('them', 672), ('t', 668), ('an', 660), ('so', 616), ('o', 590), ('then', 577), ('stephen', 570), ('if', 561), ('are', 556), ('when', 551), ('about', 542), ('which', 525), ('who', 521), ('were', 511), ('your', 496), ('this', 490), ('old', 490), ('we', 473), ('says', 473), ('do', 454), ('down', 451), ('man', 446), ('over

INFO:distributed.scheduler:Scheduler closing due to unknown reason...
INFO:distributed.scheduler:Scheduler closing all comms
